### Connecting to the Kafka server

In [ ]:
import json
import time 
import pandas as pd
from kafka import KafkaProducer

In [ ]:
def json_serializer(data):
    return json.dumps(data).encode('utf-8')

In [ ]:
server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

In [ ]:
producer.bootstrap_connected()

### Sending data to the stream

In [ ]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)

producer.flush()

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

### Sending the Trip Data

In [ ]:
df_green = pd.read_csv("../green_tripdata_2019-10.csv", sep=',')
df_green

In [ ]:
df_green_filtered = df_green[['lpep_pickup_datetime',
                              'lpep_dropoff_datetime',
                              'PULocationID',
                              'DOLocationID',
                              'passenger_count',
                              'trip_distance',
                              'tip_amount']]

In [ ]:
t0 = time.time()

topic_name = 'green-trips'

for row in df_green_filtered.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    # Sending the data
    producer.send(topic_name, value=row_dict)
    print(f"Sent: {row_dict}")
    #time.sleep(0.05)

producer.flush()

t1 = time.time()
print(f'Took {(t1 - t0):.2f} seconds')

producer.close()

### Creating the PySpark consumer


In [ ]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

In [ ]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

In [ ]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [ ]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()

In [ ]:
query.stop()

In [ ]:
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

In [ ]:
from pyspark.sql import functions as F

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")